<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scrape-Basic-Player-Stats" data-toc-modified-id="Scrape-Basic-Player-Stats-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scrape Basic Player Stats</a></span></li><li><span><a href="#Scrape-Advanced-Player-Stats" data-toc-modified-id="Scrape-Advanced-Player-Stats-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Scrape Advanced Player Stats</a></span></li><li><span><a href="#Scrape-Defensive-Player-Stats" data-toc-modified-id="Scrape-Defensive-Player-Stats-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Scrape Defensive Player Stats</a></span></li><li><span><a href="#Scrape-Player-Usage-Stats" data-toc-modified-id="Scrape-Player-Usage-Stats-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Scrape Player Usage Stats</a></span></li><li><span><a href="#Scrape-Miscellaneous-Player-Stats" data-toc-modified-id="Scrape-Miscellaneous-Player-Stats-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Scrape Miscellaneous Player Stats</a></span></li><li><span><a href="#Scrape-Player-Distance/Speed-Tracking-Stats" data-toc-modified-id="Scrape-Player-Distance/Speed-Tracking-Stats-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Scrape Player Distance/Speed Tracking Stats</a></span></li><li><span><a href="#Scrape-Player-Stats-for-Field-Goals-by-Distance" data-toc-modified-id="Scrape-Player-Stats-for-Field-Goals-by-Distance-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Scrape Player Stats for Field Goals by Distance</a></span><ul class="toc-item"><li><span><a href="#Offensive-Field-Goals-by-Distance" data-toc-modified-id="Offensive-Field-Goals-by-Distance-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Offensive Field Goals by Distance</a></span></li><li><span><a href="#Opponent-(Defensive)-Field-Goals-by-Distance" data-toc-modified-id="Opponent-(Defensive)-Field-Goals-by-Distance-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Opponent (Defensive) Field Goals by Distance</a></span></li></ul></li><li><span><a href="#Scrape-Player-Stats-for-Shots-by-Defender-Proximity" data-toc-modified-id="Scrape-Player-Stats-for-Shots-by-Defender-Proximity-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Scrape Player Stats for Shots by Defender Proximity</a></span></li><li><span><a href="#Scrape-Player-Stats-for-Shots-by-Number-of-Dribbles" data-toc-modified-id="Scrape-Player-Stats-for-Shots-by-Number-of-Dribbles-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Scrape Player Stats for Shots by Number of Dribbles</a></span></li><li><span><a href="#Scrape-Player-Stats-for-Shot-Selection-and-Efficiency" data-toc-modified-id="Scrape-Player-Stats-for-Shot-Selection-and-Efficiency-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Scrape Player Stats for Shot Selection and Efficiency</a></span></li><li><span><a href="#Scrape-Defensive-Player-Stats-(by-Opponent-Field-Goals)" data-toc-modified-id="Scrape-Defensive-Player-Stats-(by-Opponent-Field-Goals)-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Scrape Defensive Player Stats (by Opponent Field Goals)</a></span></li><li><span><a href="#Scrape-Player-Passing-and-Assist-Stats" data-toc-modified-id="Scrape-Player-Passing-and-Assist-Stats-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Scrape Player Passing and Assist Stats</a></span></li><li><span><a href="#Scrape-Player-Rebounding-Stats" data-toc-modified-id="Scrape-Player-Rebounding-Stats-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Scrape Player Rebounding Stats</a></span></li><li><span><a href="#Scrape-Player-Box-Out-Stats" data-toc-modified-id="Scrape-Player-Box-Out-Stats-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Scrape Player Box-Out Stats</a></span></li><li><span><a href="#Scrape-Player-Hustle-Stats" data-toc-modified-id="Scrape-Player-Hustle-Stats-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Scrape Player Hustle Stats</a></span></li><li><span><a href="#Merge-and-Store-Dataframes-for-All-Player-Stats" data-toc-modified-id="Merge-and-Store-Dataframes-for-All-Player-Stats-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Merge and Store Dataframes for All Player Stats</a></span></li></ul></div>

In [1]:
import numpy as np
import urllib.request
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions
import lxml.html
from lxml import etree
import re
import time
import pandas as pd
from functools import reduce
from operator import itemgetter

In [2]:
# Modify dataframe to have appropriate data types
def ConvertDataFrame(df):
    cols = df.columns.drop(['name'])
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    df['name'] = df['name'].astype('str')
    df['year'] = df['year'].astype('int')
    df = df.groupby(['name', 'year']).mean().reset_index()
    return df


In [102]:
# Scrape player data tables from NBA.com webpages, grabbing only
# the specified columns (by index) and for the specified seasons
def FetchStatsTables(urls, years, col_list):
    # Create a headless Firefox browser instance
    opt = FirefoxOptions()
    opt.add_argument("--headless")
    driver = webdriver.Firefox(options=opt)
    
    arr = []
    for i,url in enumerate(urls):
        year = years[i]
        print("Fetching player stats from the", year, "season...")
        
        driver.get(url)
        wait = WebDriverWait(driver, 30)
        wait.until(EC.presence_of_element_located((By.XPATH, "//select[contains(@class, 'stats-table-pagination__select')]")))
        sel = Select(driver.find_element_by_class_name('stats-table-pagination__select'))
        sel.select_by_visible_text("All")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        
        retries = 1
        while retries <= 3:
            try:
                wait.until(EC.presence_of_element_located((By.XPATH, "//*[@class='nba-stat-table__overflow']//table/tbody/tr")))
                break
            except TimeoutException:
                print('\nRefreshing lineup page due to timeout (retry #', retries,')...')
                driver.refresh()
                time.sleep(1)
                retries += 1

        root = lxml.html.fromstring(driver.page_source)
        results = root.xpath("//*[@class='nba-stat-table__overflow']//table/tbody/tr")
    
        # Process the table text and break into columns, 
        # stripping extraneous newline characters and inserting the season year
        counter = 0
        for result in results:
            item = result.xpath("./td//text()")
            item = [re.sub('\n +', '', x) for x in item]
            data = [x for x in item if x != '' and x != '\n']
            data = [s.strip('%') for s in data]
            if len(data) < col_list[-1]+1:
                continue
            data = list(itemgetter(*col_list)(data))
            data.insert(1, int(year))
            arr.append(data)
            counter += 1
            #print(data)
        
        print("Fetched stats for", counter, "NBA players.")
        
    driver.quit()
    #print(arr)
    return np.array(arr)


# Scrape Basic Player Stats

In [100]:
# Establish the years for which we want to fetch player data
# (for data available for 2014-15 onward)
ya = [str(n).zfill(2) for n in range(14, 20)]
yb = [str(n).zfill(2) for n in range(15, 21)]
years = [int("20"+y) for y in yb]


In [6]:
# Create URLs for the available years of NBA.com basic stats data (per 36 minutes), 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/traditional/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per36".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_basic = FetchStatsTables(urls, years, [0, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 28])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [7]:
# Convert the dataframe to one with appropriate data types
df_basic = pd.DataFrame(np_arr_basic, columns=['name', 'year', 'age', 'W', 'L', 'PTS_PT', 'FGM_PT', 'FGA_PT', 'FGP_PT', '3PM_PT', '3PA_PT', '3PP_PT', 'FTM_PT', 'FTA_PT', 'FTP_PT', 'ORB_PT', 'DRB_PT', 'TRB_PT', 'AST_PT', 'TOV_PT', 'STL_PT', 'BLK_PT', 'PF_PT', 'PM_PT'])
df_basic = ConvertDataFrame(df_basic)
print(df_basic)

                name  year   age     W     L  PTS_PT  FGM_PT  FGA_PT  FGP_PT  \
0         AJ Hammons  2017  24.0   4.0  18.0    10.6     3.7     9.3    40.5   
1           AJ Price  2015  28.0  11.0  15.0    14.8     5.7    15.2    37.2   
2       Aaron Brooks  2015  30.0  50.0  32.0    18.2     6.6    15.6    42.1   
3       Aaron Brooks  2016  31.0  36.0  33.0    16.0     6.1    15.2    40.1   
4       Aaron Brooks  2017  32.0  36.0  29.0    13.0     4.9    12.1    40.3   
...              ...   ...   ...   ...   ...     ...     ...     ...     ...   
2999    Zhaire Smith  2019  20.0   2.0   4.0    13.0     4.6    11.1    41.2   
3000         Zhou Qi  2018  22.0  14.0   4.0     6.4     1.7     9.3    18.8   
3001         Zhou Qi  2019  23.0   0.0   1.0    75.4    37.7    37.7   100.0   
3002    Zoran Dragic  2015  26.0   6.0  10.0    13.4     5.3    14.4    36.7   
3003  Zylan Cheatham  2020  24.0   1.0   1.0     3.6     1.8     7.3    25.0   

      3PM_PT  ...  FTP_PT  ORB_PT  DRB_

In [8]:
# Create URLs for the available years of NBA.com basic stats data (per 100 possessions), 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/traditional/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per100Possessions".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_basic2 = FetchStatsTables(urls, years, [0, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 28])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [9]:
df_basic2 = pd.DataFrame(np_arr_basic2, columns=['name', 'year', 'PTS_PH', 'FGM_PH', 'FGA_PH', 'FGP_PH', '3PM_PH', '3PA_PH', '3PP_PH', 'FTM_PH', 'FTA_PH', 'FTP_PH', 'ORB_PH', 'DRB_PH', 'TRB_PH', 'AST_PH', 'TOV_PH', 'STL_PH', 'BLK_PH', 'PF_PH', 'PM_PH'])
df_basic2 = ConvertDataFrame(df_basic2)
print(df_basic2)

                name  year  PTS_PH  FGM_PH  FGA_PH  FGP_PH  3PM_PH  3PA_PH  \
0         AJ Hammons  2017    14.7     5.2    12.8    40.5     1.5     3.1   
1           AJ Price  2015    21.2     8.1    21.9    37.2     2.4     9.1   
2       Aaron Brooks  2015    25.7     9.3    22.0    42.1     3.3     8.4   
3       Aaron Brooks  2016    21.8     8.4    20.9    40.1     2.9     8.2   
4       Aaron Brooks  2017    17.9     6.7    16.6    40.3     2.7     7.1   
...              ...   ...     ...     ...     ...     ...     ...     ...   
2999    Zhaire Smith  2019    16.9     5.9    14.3    41.2     2.5     6.8   
3000         Zhou Qi  2018     8.3     2.3    12.1    18.8     0.8     7.2   
3001         Zhou Qi  2019    66.7    33.3    33.3   100.0     0.0     0.0   
3002    Zoran Dragic  2015    17.8     7.0    19.1    36.7     1.9     8.9   
3003  Zylan Cheatham  2020     5.3     2.6    10.5    25.0     0.0     2.6   

      3PP_PH  FTM_PH  ...  FTP_PH  ORB_PH  DRB_PH  TRB_PH  AST_

# Scrape Advanced Player Stats

In [10]:
# Create URLs for the available years of NBA.com advanced stats data, 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/advanced/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_adv = FetchStatsTables(urls, years, [0,7,8,9,17,20,21])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [11]:
df_adv = pd.DataFrame(np_arr_adv, columns=['name', 'year', 'OFFRTG', 'DEFRTG', 'NETRTG', 'EFGP', 'PACE', 'PIE'])
df_adv = ConvertDataFrame(df_adv)
print(df_adv)

                name  year  OFFRTG  DEFRTG  NETRTG   EFGP    PACE   PIE
0         AJ Hammons  2017   101.2   101.5    -0.3   46.4   96.75   4.3
1           AJ Price  2015    94.4   104.5   -10.0   42.7   93.11   8.9
2       Aaron Brooks  2015   107.2   102.1     5.0   49.5   94.35   9.2
3       Aaron Brooks  2016   102.3   102.3     0.0   47.1   97.72   7.3
4       Aaron Brooks  2017   101.8   103.4    -1.6   48.3   96.91   6.2
...              ...   ...     ...     ...     ...    ...     ...   ...
2999    Zhaire Smith  2019   118.1   110.8     7.3   50.0  103.49   5.4
3000         Zhou Qi  2018    81.5   107.6   -26.1   21.9  102.22  -0.5
3001         Zhou Qi  2019    66.7   133.3   -66.7  100.0  150.79  33.3
3002    Zoran Dragic  2015    92.4   101.3    -8.9   41.7  100.50   3.3
3003  Zylan Cheatham  2020    78.9   132.5   -53.6   25.0   94.86  -1.3

[3004 rows x 8 columns]


# Scrape Defensive Player Stats

In [12]:
# Create URLs for the available years of NBA.com defensive stats data (per 36 minutes), 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/defense/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per36".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_def = FetchStatsTables(urls, years, [0,15,16,17,18])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [13]:
df_def = pd.DataFrame(np_arr_def, columns=['name', 'year', 'OPP_PTS_TOV_PT', 'OPP_PTS_2ND_PT', 'OPP_PTS_FB_PT', 'OPP_PTS_PAINT_PT'])
df_def = ConvertDataFrame(df_def)
print(df_def)

                name  year  OPP_PTS_TOV_PT  OPP_PTS_2ND_PT  OPP_PTS_FB_PT  \
0         AJ Hammons  2017            13.0             9.7           10.1   
1           AJ Price  2015            13.0             8.0            7.6   
2       Aaron Brooks  2015            11.5             9.9            8.8   
3       Aaron Brooks  2016            11.8             9.3           10.3   
4       Aaron Brooks  2017            11.2             8.5            8.5   
...              ...   ...             ...             ...            ...   
2999    Zhaire Smith  2019            12.4             9.8           12.4   
3000         Zhou Qi  2018            14.8             9.9            4.6   
3001         Zhou Qi  2019             0.0             0.0            0.0   
3002    Zoran Dragic  2015            14.8            12.9            4.3   
3003  Zylan Cheatham  2020            14.6            20.1           10.9   

      OPP_PTS_PAINT_PT  
0                 32.2  
1                 31.6  


In [14]:
# Create URLs for the available years of NBA.com defensive stats data (per 100 possessions), 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/defense/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per100Possessions".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_def2 = FetchStatsTables(urls, years, [0,15,16,17,18])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [15]:
df_def2 = pd.DataFrame(np_arr_def2, columns=['name', 'year', 'OPP_PTS_TOV_PH', 'OPP_PTS_2ND_PH', 'OPP_PTS_FB_PH', 'OPP_PTS_PAINT_PH'])
df_def2 = ConvertDataFrame(df_def2)
print(df_def2)

                name  year  OPP_PTS_TOV_PH  OPP_PTS_2ND_PH  OPP_PTS_FB_PH  \
0         AJ Hammons  2017            18.0            13.5           14.1   
1           AJ Price  2015            18.7            11.5           10.8   
2       Aaron Brooks  2015            16.2            13.9           12.4   
3       Aaron Brooks  2016            16.1            12.8           14.1   
4       Aaron Brooks  2017            15.4            11.6           11.7   
...              ...   ...             ...             ...            ...   
2999    Zhaire Smith  2019            16.0            12.7           16.0   
3000         Zhou Qi  2018            19.2            12.8            6.0   
3001         Zhou Qi  2019             0.0             0.0            0.0   
3002    Zoran Dragic  2015            19.7            17.2            5.7   
3003  Zylan Cheatham  2020            21.1            28.9           15.8   

      OPP_PTS_PAINT_PH  
0                 44.6  
1                 45.3  


# Scrape Player Usage Stats

In [16]:
# Create URLs for the available years of NBA.com usage stats data, 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/usage/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_usg = FetchStatsTables(urls, years, [0,8,9,10,11,12,13,14,15,16,17,18,19,20,24])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [17]:
df_usg = pd.DataFrame(np_arr_usg, columns=['name', 'year', 'PERC_FGM', 'PERC_FGA', 'PERC_3PM', 'PERC_3PA', 'PERC_FTM', 'PERC_FTA', 'PERC_ORB', 'PERC_DRB', 'PERC_TRB', 'PERC_AST', 'PERC_TOV', 'PERC_STL', 'PERC_BLK', 'PERC_PTS'])
df_usg = ConvertDataFrame(df_usg)
print(df_usg)

                name  year  PERC_FGM  PERC_FGA  PERC_3PM  PERC_3PA  PERC_FTM  \
0         AJ Hammons  2017      14.0      14.7      12.8       8.8      18.0   
1           AJ Price  2015      22.0      24.2      31.3      37.0      20.0   
2       Aaron Brooks  2015      24.4      25.7      35.0      33.9      17.8   
3       Aaron Brooks  2016      21.7      23.4      28.9      30.9      14.4   
4       Aaron Brooks  2017      17.3      18.7      30.4      29.6      11.5   
...              ...   ...       ...       ...       ...       ...       ...   
2999    Zhaire Smith  2019      12.4      15.7      20.0      23.2      25.0   
3000         Zhou Qi  2018       7.8      15.0       6.7      16.4      25.0   
3001         Zhou Qi  2019     100.0     100.0       0.0       0.0       0.0   
3002    Zoran Dragic  2015      20.4      22.2      23.1      27.5      12.5   
3003  Zylan Cheatham  2020       9.1      12.9       0.0       7.1       0.0   

      PERC_FTA  PERC_ORB  PERC_DRB  PER

# Scrape Miscellaneous Player Stats

In [18]:
# Create URLs for the available years of NBA.com miscellaneous stats data (per 36 minutes), 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/misc/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per36".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_misc = FetchStatsTables(urls, years, [0,7,8,9,10,16,18])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [19]:
df_misc = pd.DataFrame(np_arr_misc, columns=['name', 'year', 'PTS_OFF_TOV_PT', 'SEC_CHANCE_PTS_PT', 'FB_PTS_PT', 'PTS_PAINT_PT', 'BLK_ATTEMPT_PT', 'PF_DRAWN_PT'])
df_misc = ConvertDataFrame(df_misc)
print(df_misc)

                name  year  PTS_OFF_TOV_PT  SEC_CHANCE_PTS_PT  FB_PTS_PT  \
0         AJ Hammons  2017             1.3                1.5        0.4   
1           AJ Price  2015             3.1                1.4        0.6   
2       Aaron Brooks  2015             1.3                1.0        1.7   
3       Aaron Brooks  2016             1.2                0.5        1.3   
4       Aaron Brooks  2017             1.6                1.2        1.0   
...              ...   ...             ...                ...        ...   
2999    Zhaire Smith  2019             2.6                1.3        0.7   
3000         Zhou Qi  2018             0.3                0.6        1.2   
3001         Zhou Qi  2019             0.0                0.0        0.0   
3002    Zoran Dragic  2015             6.7                1.0        6.2   
3003  Zylan Cheatham  2020             0.0                0.0        0.0   

      PTS_PAINT_PT  BLK_ATTEMPT_PT  PF_DRAWN_PT  
0              2.2             1.1   

In [20]:
# Create URLs for the available years of NBA.com miscellaneous stats data (per 100 possessions), 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/misc/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per100Possessions".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_misc2 = FetchStatsTables(urls, years, [0,7,8,9,10,16,18])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.


In [21]:
df_misc2 = pd.DataFrame(np_arr_misc2, columns=['name', 'year', 'PTS_OFF_TOV_PH', 'SEC_CHANCE_PTS_PH', 'FB_PTS_PH', 'PTS_PAINT_PH', 'BLK_ATTEMPT_PH', 'PF_DRAWN_PH'])
df_misc2 = ConvertDataFrame(df_misc2)
print(df_misc2)

                name  year  PTS_OFF_TOV_PH  SEC_CHANCE_PTS_PH  FB_PTS_PH  \
0         AJ Hammons  2017             1.8                2.1        0.6   
1           AJ Price  2015             4.5                2.1        0.8   
2       Aaron Brooks  2015             1.8                1.3        2.4   
3       Aaron Brooks  2016             1.7                0.7        1.8   
4       Aaron Brooks  2017             2.2                1.7        1.3   
...              ...   ...             ...                ...        ...   
2999    Zhaire Smith  2019             3.4                1.7        0.8   
3000         Zhou Qi  2018             0.4                0.8        1.5   
3001         Zhou Qi  2019             0.0                0.0        0.0   
3002    Zoran Dragic  2015             8.9                1.3        8.3   
3003  Zylan Cheatham  2020             0.0                0.0        0.0   

      PTS_PAINT_PH  BLK_ATTEMPT_PH  PF_DRAWN_PH  
0              3.1             1.5   

# Scrape Player Distance/Speed Tracking Stats

In [34]:
# Create URLs for the available years of NBA.com distance/speed stats data, 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/speed-distance/?sort=PLAYER_NAME&dir=-1&Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_dist = FetchStatsTables(urls, years, [0,6,7,8,9,10,11,12])


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 477 NBA players.


In [35]:
df_dist = pd.DataFrame(np_arr_dist, columns=['name', 'year', 'DIST_FEET_PG', 'DIST_M_PG', 'DIST_M_OFF', 'DIST_M_DEF', 'AVG_SPEED', 'AVG_SPEED_OFF', 'AVG_SPEED_DEF'])
df_dist = ConvertDataFrame(df_dist)
print(df_dist)

                name  year  DIST_FEET_PG  DIST_M_PG  DIST_M_OFF  DIST_M_DEF  \
0         AJ Hammons  2017        2571.2       0.49        0.25        0.23   
1           AJ Price  2015        4690.0       0.89        0.50        0.39   
2       Aaron Brooks  2015        8351.7       1.58        0.87        0.71   
3       Aaron Brooks  2016        5779.6       1.09        0.58        0.51   
4       Aaron Brooks  2017        4953.5       0.94        0.51        0.43   
...              ...   ...           ...        ...         ...         ...   
2996    Zhaire Smith  2019        7941.8       1.50        0.84        0.67   
2997         Zhou Qi  2018        2525.6       0.48        0.26        0.22   
2998         Zhou Qi  2019         385.0       0.07        0.03        0.04   
2999    Zoran Dragic  2015        1794.4       0.34        0.18        0.16   
3000  Zylan Cheatham  2020        4062.5       0.77        0.40        0.36   

      AVG_SPEED  AVG_SPEED_OFF  AVG_SPEED_DEF  
0  

# Scrape Player Stats for Field Goals by Distance

## Offensive Field Goals by Distance

In [36]:
# Field goals by distance

# Close range (5 ft)
#https://stats.nba.com/players/shooting/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/shooting/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_fg5ft = FetchStatsTables(urls, years, [0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
df_fg5ft = pd.DataFrame(np_arr_fg5ft, columns=['name', 'year', 'FGM_05FT', 'FGA_05FT', 'FGP_05FT', 'FGM_59FT', 'FGA_59FT', 'FGP_59FT', 'FGM_1014FT', 'FGA_1014FT', 'FGP_1014FT', 'FGM_1519FT', 'FGA_1519FT', 'FGP_1519FT', 'FGM_2024FT', 'FGA_2024FT', 'FGP_2024FT', 'FGM_2529FT', 'FGA_2529FT', 'FGP_2529FT'])
df_fg5ft = ConvertDataFrame(df_fg5ft)

# Farther range (8 ft)
#https://stats.nba.com/players/shooting/?Season=2014-15&SeasonType=Regular%20Season&DistanceRange=8ft%20Range
urls = [ "https://stats.nba.com/players/shooting/?Season=20{0}-{1}&SeasonType=Regular%20Season&DistanceRange=8ft%20Range".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_fg8ft = FetchStatsTables(urls, years, [0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_fg8ft = pd.DataFrame(np_arr_fg8ft, columns=['name', 'year', 'FGM_08FT', 'FGA_08FT', 'FGP_08FT', 'FGM_816FT', 'FGA_816FT', 'FGP_816FT', 'FGM_1624FT', 'FGA_1624FT', 'FGP_1624FT', 'FGM_GT24FT', 'FGA_GT24FT', 'FGP_GT24FT', 'FGM_BC', 'FGA_BC', 'FGP_BC'])
df_fg8ft = ConvertDataFrame(df_fg8ft)

# By zone
#https://stats.nba.com/players/shooting/?Season=2014-15&SeasonType=Regular%20Season&DistanceRange=By%20Zone
urls = [ "https://stats.nba.com/players/shooting/?Season=20{0}-{1}&SeasonType=Regular%20Season&DistanceRange=By%20Zone".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_fgzone = FetchStatsTables(urls, years, [0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
df_fgzone = pd.DataFrame(np_arr_fgzone, columns=['name', 'year', 'FGM_RA', 'FGA_RA', 'FGP_RA', 'FGM_NONRA', 'FGA_NONRA', 'FGP_NONRA', 'FGM_MR', 'FGA_MR', 'FGP_MR', 'FGM_LC', 'FGA_LC', 'FGP_LC', 'FGM_RC', 'FGA_RC', 'FGP_RC', 'FGM_AB', 'FGA_AB', 'FGP_AB'])
df_fgzone = ConvertDataFrame(df_fgzone)
print(df_fgzone)


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
Fetching player stats from t

## Opponent (Defensive) Field Goals by Distance

In [37]:
# Opponent field goals by distance
# Close range (5 ft)
#https://stats.nba.com/players/opponent-shooting/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/opponent-shooting/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_ofg5ft = FetchStatsTables(urls, years, [0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
df_ofg5ft = pd.DataFrame(np_arr_ofg5ft, columns=['name', 'year', 'OPP_FGM_05FT', 'OPP_FGA_05FT', 'OPP_FGP_05FT', 'OPP_FGM_59FT', 'OPP_FGA_59FT', 'OPP_FGP_59FT', 'OPP_FGM_1014FT', 'OPP_FGA_1014FT', 'OPP_FGP_1014FT', 'OPP_FGM_1519FT', 'OPP_FGA_1519FT', 'OPP_FGP_1519FT', 'OPP_FGM_2024FT', 'OPP_FGA_2024FT', 'OPP_FGP_2024FT', 'OPP_FGM_2529FT', 'OPP_FGA_2529FT', 'OPP_FGP_2529FT'])
df_ofg5ft = ConvertDataFrame(df_ofg5ft)
    
# Farther range (8 ft)
#https://stats.nba.com/players/opponent-shooting/?Season=2014-15&SeasonType=Regular%20Season&DistanceRange=8ft%20Range
urls = [ "https://stats.nba.com/players/opponent-shooting/?Season=20{0}-{1}&SeasonType=Regular%20Season&DistanceRange=8ft%20Range".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_ofg8ft = FetchStatsTables(urls, years, [0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_ofg8ft = pd.DataFrame(np_arr_ofg8ft, columns=['name', 'year', 'OPP_FGM_08FT', 'OPP_FGA_08FT', 'OPP_FGP_08FT', 'OPP_FGM_816FT', 'OPP_FGA_816FT', 'OPP_FGP_816FT', 'OPP_FGM_1624FT', 'OPP_FGA_1624FT', 'OPP_FGP_1624FT', 'OPP_FGM_GT24FT', 'OPP_FGA_GT24FT', 'OPP_FGP_GT24FT', 'OPP_FGM_BC', 'OPP_FGA_BC', 'OPP_FGP_BC'])
df_ofg8ft = ConvertDataFrame(df_ofg8ft)

# By zone
#https://stats.nba.com/players/opponent-shooting/?Season=2014-15&SeasonType=Regular%20Season&DistanceRange=By%20Zone
urls = [ "https://stats.nba.com/players/opponent-shooting/?Season=20{0}-{1}&SeasonType=Regular%20Season&DistanceRange=By%20Zone".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_ofgzone = FetchStatsTables(urls, years, [0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
df_ofgzone = pd.DataFrame(np_arr_ofgzone, columns=['name', 'year', 'OPP_FGM_RA', 'OPP_FGA_RA', 'OPP_FGP_RA', 'OPP_FGM_NONRA', 'OPP_FGA_NONRA', 'OPP_FGP_NONRA', 'OPP_FGM_MR', 'OPP_FGA_MR', 'OPP_FGP_MR', 'OPP_FGM_LC', 'OPP_FGA_LC', 'OPP_FGP_LC', 'OPP_FGM_RC', 'OPP_FGA_RC', 'OPP_FGP_RC', 'OPP_FGM_AB', 'OPP_FGA_AB', 'OPP_FGP_AB'])
df_ofgzone = ConvertDataFrame(df_ofgzone)
print(df_ofgzone)


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
Fetching player stats from t

# Scrape Player Stats for Shots by Defender Proximity

In [64]:
# Shots by distance to closest defender

# Very tight (0-2 ft)
#https://stats.nba.com/players/shots-closest-defender/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/shots-closest-defender/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_vtdef = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_vtdef = pd.DataFrame(np_arr_vtdef, columns=['name', 'year', 'FG_FREQ_DEF02FT', 'FGM_DEF02FT', 'FGA_DEF02FT', 'FGP_DEF02FT', 'EFGP_DEF02FT', '2PT_FREQ_DEF02FT', '2PM_DEF02FT', '2PA_DEF02FT', '2PP_DEF02FT', '3PT_FREQ_DEF02FT', '3PM_DEF02FT', '3PA_DEF02FT', '3PP_DEF02FT'])
df_vtdef = ConvertDataFrame(df_vtdef)
print(df_vtdef)

# Tight (2-4 ft)
#https://stats.nba.com/players/shots-closest-defender/?Season=2014-15&SeasonType=Regular%20Season&CloseDefDistRange=2-4%20Feet%20-%20Tight
urls = [ "https://stats.nba.com/players/shots-closest-defender/?Season=20{0}-{1}&SeasonType=Regular%20Season&CloseDefDistRange=2-4%20Feet%20-%20Tight".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_tdef = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_tdef = pd.DataFrame(np_arr_tdef, columns=['name', 'year', 'FG_FREQ_DEF24FT', 'FGM_DEF24FT', 'FGA_DEF24FT', 'FGP_DEF24FT', 'EFGP_DEF24FT', '2PT_FREQ_DEF24FT', '2PM_DEF24FT', '2PA_DEF24FT', '2PP_DEF24FT', '3PT_FREQ_DEF24FT', '3PM_DEF24FT', '3PA_DEF24FT', '3PP_DEF24FT'])
df_tdef = ConvertDataFrame(df_tdef)

# Open (4-6 ft)
#https://stats.nba.com/players/shots-closest-defender/?Season=2014-15&SeasonType=Regular%20Season&CloseDefDistRange=4-6%20Feet%20-%20Open
urls = [ "https://stats.nba.com/players/shots-closest-defender/?Season=20{0}-{1}&SeasonType=Regular%20Season&CloseDefDistRange=4-6%20Feet%20-%20Open".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_opendef = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_opendef = pd.DataFrame(np_arr_opendef, columns=['name', 'year', 'FG_FREQ_DEF46FT', 'FGM_DEF46FT', 'FGA_DEF46FT', 'FGP_DEF46FT', 'EFGP_DEF46FT', '2PT_FREQ_DEF46FT', '2PM_DEF46FT', '2PA_DEF46FT', '2PP_DEF46FT', '3PT_FREQ_DEF46FT', '3PM_DEF46FT', '3PA_DEF46FT', '3PP_DEF46FT'])
df_opendef = ConvertDataFrame(df_opendef)

# Wide open (6+ ft)
#https://stats.nba.com/players/shots-closest-defender/?Season=2014-15&SeasonType=Regular%20Season&CloseDefDistRange=6%2B%20Feet%20-%20Wide%20Open
urls = [ "https://stats.nba.com/players/shots-closest-defender/?Season=20{0}-{1}&SeasonType=Regular%20Season&CloseDefDistRange=6%2B%20Feet%20-%20Wide%20Open".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_wodef = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_wodef = pd.DataFrame(np_arr_wodef, columns=['name', 'year', 'FG_FREQ_DEFGT6FT', 'FGM_DEFGT6FT', 'FGA_DEFGT6FT', 'FGP_DEFGT6FT', 'EFGP_DEFGT6FT', '2PT_FREQ_DEFGT6FT', '2PM_DEFGT6FT', '2PA_DEFGT6FT', '2PP_DEFGT6FT', '3PT_FREQ_DEFGT6FT', '3PM_DEFGT6FT', '3PA_DEFGT6FT', '3PP_DEFGT6FT'])
df_wodef = ConvertDataFrame(df_wodef)


Fetching player stats from the 2017 season...
Fetched stats for 469 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 490 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 499 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 429 NBA players.
               name  year  FG_FREQ_DEF02FT  FGM_DEF02FT  FGA_DEF02FT  \
0        AJ Hammons  2017              8.1          0.1          0.1   
1      Aaron Brooks  2017             17.4          0.3          0.8   
2      Aaron Brooks  2018             12.3          0.1          0.3   
3      Aaron Gordon  2017             13.0          0.8          1.3   
4      Aaron Gordon  2018              8.2          0.7          1.2   
...             ...   ...              ...          ...          ...   
1882  Zaza Pachulia  2017             31.5          0.7          1.2   
1883  Zaza Pachulia  2018             15.9          0.3          0.5   
1884  Zaza Pachulia  2019   

# Scrape Player Stats for Shots by Number of Dribbles

In [65]:
# Shots based on # dribbles
# 0 dribbles
#https://stats.nba.com/players/shots-dribbles/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/shots-dribbles/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_0drib = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_0drib = pd.DataFrame(np_arr_0drib, columns=['name', 'year', 'FG_FREQ_0DRIB', 'FGM_0DRIB', 'FGA_0DRIB', 'FGP_0DRIB', 'EFGP_0DRIB', '2PT_FREQ_0DRIB', '2PM_0DRIB', '2PA_0DRIB', '2PP_0DRIB', '3PT_FREQ_0DRIB', '3PM_0DRIB', '3PA_0DRIB', '3PP_0DRIB'])
df_0drib = ConvertDataFrame(df_0drib)
print(df_0drib)
# 1
#https://stats.nba.com/players/shots-dribbles/?Season=2014-15&SeasonType=Regular%20Season&DribbleRange=1%20Dribble
urls = [ "https://stats.nba.com/players/shots-dribbles/?Season=20{0}-{1}&SeasonType=Regular%20Season&DribbleRange=1%20Dribble".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_1drib = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_1drib = pd.DataFrame(np_arr_1drib, columns=['name', 'year', 'FG_FREQ_1DRIB', 'FGM_1DRIB', 'FGA_1DRIB', 'FGP_1DRIB', 'EFGP_1DRIB', '2PT_FREQ_1DRIB', '2PM_1DRIB', '2PA_1DRIB', '2PP_1DRIB', '3PT_FREQ_1DRIB', '3PM_1DRIB', '3PA_1DRIB', '3PP_1DRIB'])
df_1drib = ConvertDataFrame(df_1drib)
# 2
#https://stats.nba.com/players/shots-dribbles/?Season=2014-15&SeasonType=Regular%20Season&DribbleRange=2%20Dribbles
urls = [ "https://stats.nba.com/players/shots-dribbles/?Season=20{0}-{1}&SeasonType=Regular%20Season&DribbleRange=2%20Dribbles".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_2drib = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_2drib = pd.DataFrame(np_arr_2drib, columns=['name', 'year', 'FG_FREQ_2DRIB', 'FGM_2DRIB', 'FGA_2DRIB', 'FGP_2DRIB', 'EFGP_2DRIB', '2PT_FREQ_2DRIB', '2PM_2DRIB', '2PA_2DRIB', '2PP_2DRIB', '3PT_FREQ_2DRIB', '3PM_2DRIB', '3PA_2DRIB', '3PP_2DRIB'])
df_2drib = ConvertDataFrame(df_2drib)
# 3-6
#https://stats.nba.com/players/shots-dribbles/?Season=2014-15&SeasonType=Regular%20Season&DribbleRange=3-6%20Dribbles
urls = [ "https://stats.nba.com/players/shots-dribbles/?Season=20{0}-{1}&SeasonType=Regular%20Season&DribbleRange=3-6%20Dribbles".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_3drib = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_3drib = pd.DataFrame(np_arr_3drib, columns=['name', 'year', 'FG_FREQ_36DRIB', 'FGM_36DRIB', 'FGA_36DRIB', 'FGP_36DRIB', 'EFGP_36DRIB', '2PT_FREQ_36DRIB', '2PM_36DRIB', '2PA_36DRIB', '2PP_36DRIB', '3PT_FREQ_36DRIB', '3PM_36DRIB', '3PA_36DRIB', '3PP_36DRIB'])
df_3drib = ConvertDataFrame(df_3drib)
# 7+
#https://stats.nba.com/players/shots-dribbles/?Season=2014-15&SeasonType=Regular%20Season&DribbleRange=7%2B%20Dribbles
urls = [ "https://stats.nba.com/players/shots-dribbles/?Season=20{0}-{1}&SeasonType=Regular%20Season&DribbleRange=7%2B%20Dribbles".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_7drib = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16,17])
df_7drib = pd.DataFrame(np_arr_7drib, columns=['name', 'year', 'FG_FREQ_GT7DRIB', 'FGM_GT7DRIB', 'FGA_GT7DRIB', 'FGP_GT7DRIB', 'EFGP_GT7DRIB', '2PT_FREQ_GT7DRIB', '2PM_GT7DRIB', '2PA_GT7DRIB', '2PP_GT7DRIB', '3PT_FREQ_GT7DRIB', '3PM_GT7DRIB', '3PA_GT7DRIB', '3PP_GT7DRIB'])
df_7drib = ConvertDataFrame(df_7drib)
    

Fetching player stats from the 2017 season...
Fetched stats for 484 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 526 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 523 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 464 NBA players.
                name  year  FG_FREQ_0DRIB  FGM_0DRIB  FGA_0DRIB  FGP_0DRIB  \
0         AJ Hammons  2017           78.4        0.7        1.3       55.2   
1       Aaron Brooks  2017           27.9        0.5        1.2       41.6   
2       Aaron Brooks  2018           23.1        0.3        0.5       53.3   
3       Aaron Gordon  2017           49.8        2.7        5.1       52.6   
4       Aaron Gordon  2018           48.0        3.4        6.7       50.6   
...              ...   ...            ...        ...        ...        ...   
1992   Zaza Pachulia  2019           66.8        0.8        1.9       44.2   
1993    Zhaire Smith  2019           52.9        1.2

# Scrape Player Stats for Shot Selection and Efficiency

In [45]:
# Shooting efficiency (by shot type)
#https://stats.nba.com/players/shooting-efficiency/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/shooting-efficiency/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_shots = FetchStatsTables(urls, years, [0,7,8,9,10,11,12,13,14,15,16,17,18])
df_shots = pd.DataFrame(np_arr_shots, columns=['name', 'year', 'PTS_DRIVE', 'FGP_DRIVE', 'PTS_CANDS', 'FGP_CANDS', 'PTS_PU', 'FGP_PU', 'PTS_PT_TOUCH', 'FGP_PT_TOUCH', 'PTS_POST_TOUCH', 'FGP_POST_TOUCH', 'PTS_ELB_TOUCH', 'FGP_ELB_TOUCH'])
df_shots = ConvertDataFrame(df_shots)
print(df_shots)


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
                name  year  PTS_DRIVE  FGP_DRIVE  PTS_CANDS  FGP_CANDS  \
0         AJ Hammons  2017        0.0        0.0        1.3       50.0   
1           AJ Price  2015        1.6       48.6        1.2       32.3   
2       Aaron Brooks  2015        4.9       46.5        2.6       42.3   
3       Aaron Brooks  2016        2.9       43.1        1.6       40.9   
4       Aaron Brooks  2017        1.7       43.0        1.3       39.0   
...              ...   ...        ...        ...        ...        ...

In [46]:
# Catch and shoot
#https://stats.nba.com/players/catch-shoot/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/catch-shoot/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_cands = FetchStatsTables(urls, years, [0,5,6,8,9])
df_cands = pd.DataFrame(np_arr_cands, columns=['name', 'year', 'FGM_CANDS', 'FGA_CANDS', '3PM_CANDS', '3PA_CANDS'])
df_cands = ConvertDataFrame(df_cands)
print(df_cands)


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
                name  year  FGM_CANDS  FGA_CANDS  3PM_CANDS  3PA_CANDS
0         AJ Hammons  2017        0.5        1.0        0.2        0.5
1           AJ Price  2015        0.4        1.2        0.4        1.1
2       Aaron Brooks  2015        0.8        1.9        0.8        1.9
3       Aaron Brooks  2016        0.5        1.2        0.5        1.2
4       Aaron Brooks  2017        0.5        1.1        0.4        1.0
...              ...   ...        ...        ...        ...        ...
2999    Zhaire Sm

In [47]:
# Drives
#https://stats.nba.com/players/drives/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/drives/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_drives = FetchStatsTables(urls, years, [0,6,7,8,10,11,12,14,15,16,17,18,19,20,21,22])
df_drives = pd.DataFrame(np_arr_drives, columns=['name', 'year', 'DRIVES_PG', 'FGM_DRIVE', 'FGA_DRIVE', 'FTM_DRIVE', 'FTA_DRIVE', 'FTP_DRIVE', 'PTSP_DRIVE', 'PASS_DRIVE', 'PASSP_DRIVE', 'AST_DRIVE', 'ASTP_DRIVE', 'TOV_DRIVE', 'TOVP_DRIVE', 'PF_DRIVE', 'PFP_DRIVE'])
df_drives = ConvertDataFrame(df_drives)
print(df_drives)


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
                name  year  DRIVES_PG  FGM_DRIVE  FGA_DRIVE  FTM_DRIVE  \
0         AJ Hammons  2017        0.1        0.0        0.0        0.0   
1           AJ Price  2015        3.9        0.7        1.4        0.2   
2       Aaron Brooks  2015       10.4        2.0        4.3        0.8   
3       Aaron Brooks  2016        6.2        1.3        3.1        0.2   
4       Aaron Brooks  2017        4.2        0.7        1.7        0.2   
...              ...   ...        ...        ...        ...        ...

# Scrape Defensive Player Stats (by Opponent Field Goals)

In [103]:
# Player defense

# Overall
#https://stats.nba.com/players/defense-dash-overall/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/defense-dash-overall/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_odef = FetchStatsTables(urls, years, [0,3,7,8,9,11])
df_odef = pd.DataFrame(np_arr_odef, columns=['name', 'year', 'red_pos', 'DFGM_PG', 'DFGA_PG', 'DFGP_PG', 'DPCT_DIFF_PG'])
# For this dataframe, there is an extra string 'red_pos' that requires 
# custom treatment of type conversions
#df_odef = ConvertDataFrame(df_odef)

cols = df_odef.columns.drop(['name', 'red_pos'])
df_odef[cols] = df_odef[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df_odef['name'] = df_odef['name'].astype('str')
df_odef['red_pos'] = df_odef['red_pos'].astype('str')
df_odef['year'] = df_odef['year'].astype('int')
df_odef = df_odef.groupby(['name', 'year', 'red_pos']).mean().reset_index()


Fetching player stats from the 2015 season...
Fetched stats for 491 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 473 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 484 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 536 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 524 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 477 NBA players.


In [48]:
# Player defense (continued...)

# 2 pt.
#https://stats.nba.com/players/defense-dash-2pt/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/defense-dash-2pt/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_2ptdef = FetchStatsTables(urls, years, [0,6,7,8,9,11])
df_2ptdef = pd.DataFrame(np_arr_2ptdef, columns=['name', 'year', 'DFG_2PT_FREQ_PG', 'DFGM_2PT_PG', 'DFGA_2PT_PG', 'DFGP_2PT_PG', 'DPCT_DIFF_2PT_PG'])
df_2ptdef = ConvertDataFrame(df_2ptdef)
    
# 3 pt.
#https://stats.nba.com/players/defense-dash-3pt/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/defense-dash-3pt/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_3ptdef = FetchStatsTables(urls, years, [0,6,7,8,9,11])
df_3ptdef = pd.DataFrame(np_arr_3ptdef, columns=['name', 'year', 'DFG_3PT_FREQ_PG', 'DFGM_3PT_PG', 'DFGA_3PT_PG', 'DFGP_3PT_PG', 'DPCT_DIFF_3PT_PG'])
df_3ptdef = ConvertDataFrame(df_3ptdef)

# < 6 ft.
#https://stats.nba.com/players/defense-dash-lt6/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/defense-dash-lt6/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_odeflt6 = FetchStatsTables(urls, years, [0,6,7,8,9,11])
df_odeflt6 = pd.DataFrame(np_arr_odeflt6, columns=['name', 'year', 'DFG_LT6FT_FREQ_PG', 'DFGM_LT6FT_PG', 'DFGA_LT6FT_PG', 'DFGP_LT6FT_PG', 'DPCT_DIFF_LT6FT_PG'])
df_odeflt6 = ConvertDataFrame(df_odeflt6)

# < 10 ft.
#https://stats.nba.com/players/defense-dash-lt10/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/defense-dash-lt10/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_odeflt10 = FetchStatsTables(urls, years, [0,6,7,8,9,11])
df_odeflt10 = pd.DataFrame(np_arr_odeflt10, columns=['name', 'year', 'DFG_LT10FT_FREQ_PG', 'DFGM_LT10FT_PG', 'DFGA_LT10FT_PG', 'DFGP_LT10FT_PG', 'DPCT_DIFF_LT10FT_PG'])
df_odeflt10 = ConvertDataFrame(df_odeflt10)
    
# > 15 ft.
#https://stats.nba.com/players/defense-dash-gt15/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/defense-dash-gt15/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_odefgt15 = FetchStatsTables(urls, years, [0,6,7,8,9,11])
df_odefgt15 = pd.DataFrame(np_arr_odefgt15, columns=['name', 'year', 'DFG_GT15FT_FREQ_PG', 'DFGM_GT15FT_PG', 'DFGA_GT15FT_PG', 'DFGP_GT15FT_PG', 'DPCT_DIFF_GT15FT_PG'])
df_odefgt15 = ConvertDataFrame(df_odefgt15)

# Defense at the rim
#https://stats.nba.com/players/defensive-impact/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/defensive-impact/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_defrim = FetchStatsTables(urls, years, [0,9,10,11])
df_defrim = pd.DataFrame(np_arr_defrim, columns=['name', 'year', 'DFGM_RIM_PG', 'DFGA_RIM_PG', 'DFGP_RIM_PG'])
df_defrim = ConvertDataFrame(df_defrim)

Fetching player stats from the 2015 season...
Fetched stats for 491 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 473 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 484 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 536 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 524 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 475 NBA players.
Fetching player stats from the 2015 season...
Fetched stats for 490 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 473 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 484 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 531 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 522 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 470 NBA players.
Fetching player stats from t

# Scrape Player Passing and Assist Stats

In [67]:
# Passing
#https://stats.nba.com/players/passing/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/passing/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_pass = FetchStatsTables(urls, years, [0,6,7,9,10,11,12,13,14])
df_pass = pd.DataFrame(np_arr_pass, columns=['name', 'year', 'PASS_MADE_PG', 'PASS_REC_PG', 'SEC_AST_PG', 'POT_AST_PG', 'AST_PTS_PG', 'AST_ADJ_PG', 'AST_TO_PASS_PERC', 'AST_TO_PASS_ADJ_PERC'])
df_pass = ConvertDataFrame(df_pass)


Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 482 NBA players.


# Scrape Player Rebounding Stats

In [50]:
# Rebounds
#https://stats.nba.com/players/rebounding/?Season=2014-15&SeasonType=Regular%20Season
urls = [ "https://stats.nba.com/players/rebounding/?Season=20{0}-{1}&SeasonType=Regular%20Season".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_rebdet = FetchStatsTables(urls, years, [0,7,8,9,10,11,12,13])
df_rebdet = pd.DataFrame(np_arr_rebdet, columns=['name', 'year', 'CONT_REB_PG', 'CONT_REB_PERC', 'REB_CHANCES_PG', 'REB_CHANCE_PERC', 'DEF_REB_CHANCES', 'ADJ_REB_CHANCE_PERC', 'AVG_REB_DIST'])
df_rebdet = ConvertDataFrame(df_rebdet)


Fetching player stats from the 2015 season...
Fetched stats for 492 NBA players.
Fetching player stats from the 2016 season...
Fetched stats for 476 NBA players.
Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 477 NBA players.


# Scrape Player Box-Out Stats

In [68]:
# Establish the years for which we want to fetch player data
# (for data available for 2016-17 onward)
ya = [str(n).zfill(2) for n in range(17, 20)]
yb = [str(n).zfill(2) for n in range(18, 21)]
years = [int("20"+y) for y in yb]

# Box-outs
#https://stats.nba.com/players/box-outs/?Season=2017-18&SeasonType=Regular%20Season&PerMode=Per36
urls = [ "https://stats.nba.com/players/box-outs/?Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per36".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_boxout = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13])
df_boxout = pd.DataFrame(np_arr_boxout, columns=['name', 'year', 'BOX_OUTS_PT', 'OFF_BO_PT', 'DEF_BO_PT', 'TEAM_REB_BO_PT', 'PLAYER_REB_BO_PT', 'OFF_BO_PERC', 'DEF_BO_PERC', 'TEAM_REB_PERC_BO', 'PLAYER_REB_PERC_BO'])
df_boxout = ConvertDataFrame(df_boxout)


Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 482 NBA players.


# Scrape Player Hustle Stats

In [52]:
# Establish the years for which we want to fetch player data
# (for data available for 2016-17 onward)
ya = [str(n).zfill(2) for n in range(16, 20)]
yb = [str(n).zfill(2) for n in range(17, 21)]
years = [int("20"+y) for y in yb]

# Create URLs for the available years of NBA.com hustle stats data (per 36 minutes), 
# fetch the data in 2D array format, and put into a Pandas dataframe
urls = [ "https://stats.nba.com/players/hustle/?Season=20{0}-{1}&SeasonType=Regular%20Season&PerMode=Per36&sort=PLAYER_NAME&dir=-1".format(ya[i], yb[i]) for i in range(len(ya)) ]
np_arr_hustle = FetchStatsTables(urls, years, [0,5,6,7,8,9,10,11,12,13,14,15,16])
df_hustle = pd.DataFrame(np_arr_hustle, columns=['name', 'year', 'SCREEN_AST_PT', 'SCREEN_AST_PTS_PT', 'DEFL_PT', 'OFF_LB_RECOV_PT', 'DEF_LB_RECOV_PT', 'LB_RECOV_PT', 'PERC_OFF_LB_RECOV_PT', 'PERC_DEF_LB_RECOV_PT', 'CHARGE_DRAWN_PT', 'CONTESTS_2PT_PT', 'CONTESTS_3PT_PT', 'CONTESTS_PT'])
df_hustle = ConvertDataFrame(df_hustle)
print(df_hustle)


Fetching player stats from the 2017 season...
Fetched stats for 486 NBA players.
Fetching player stats from the 2018 season...
Fetched stats for 540 NBA players.
Fetching player stats from the 2019 season...
Fetched stats for 530 NBA players.
Fetching player stats from the 2020 season...
Fetched stats for 480 NBA players.
                name  year  SCREEN_AST_PT  SCREEN_AST_PTS_PT  DEFL_PT  \
0         AJ Hammons  2017            4.9               11.3      1.1   
1       Aaron Brooks  2017            0.6                1.4      2.0   
2       Aaron Brooks  2018            0.2                0.4      1.9   
3       Aaron Gordon  2017            0.6                1.4      1.8   
4       Aaron Gordon  2018            1.0                2.1      1.5   
...              ...   ...            ...                ...      ...   
2031   Zaza Pachulia  2019            5.6               12.8      2.9   
2032    Zhaire Smith  2019            0.3                0.6      1.3   
2033         Zhou Q

# Merge and Store Dataframes for All Player Stats

In [109]:
# Perform an outer merge of all dataframes, since we would like to 
# keep all information up front, regardless of year/season

df = pd.merge(df_basic, df_basic2, on=['name', 'year'], how='outer')
df = pd.merge(df, df_adv, on=['name', 'year'], how='outer')
df = pd.merge(df, df_def, on=['name', 'year'], how='outer')
df = pd.merge(df, df_def2, on=['name', 'year'], how='outer')
df = pd.merge(df, df_usg, on=['name', 'year'], how='outer')
df = pd.merge(df, df_misc, on=['name', 'year'], how='outer')
df = pd.merge(df, df_misc2, on=['name', 'year'], how='outer')
df = pd.merge(df, df_dist, on=['name', 'year'], how='outer')

###########################################
# Field goals by distance

# Close range (5 ft)
# Farther range (8 ft)
# By zone

# Opponent field goals by distance

# Close range (5 ft)
# Farther range (8 ft)
# By zone

# Shots by distance to closest defender

# Very tight (0-2 ft)
# Tight (2-4 ft)
# Open (4-6 ft)
# Wide open (6+ ft)

# Shots based on # dribbles
# 0 dribbles
# 1
# 2
# 3-6
# 7+
    
# Catch and shoot
    
# Drives
    
# Shooting efficiency (by shot type)

# Player defense

# Overall
# 2 pt.
# 3 pt.
# Defense at the rim

# < 6 ft.
# < 10 ft.    
# > 15 ft.

# Passing
    
# Rebounds

# Box-outs
df = pd.merge(df, df_fg5ft, on=['name', 'year'], how='outer')
df = pd.merge(df, df_fg8ft, on=['name', 'year'], how='outer')
df = pd.merge(df, df_fgzone, on=['name', 'year'], how='outer')
df = pd.merge(df, df_ofg5ft, on=['name', 'year'], how='outer')
df = pd.merge(df, df_ofg8ft, on=['name', 'year'], how='outer')
df = pd.merge(df, df_ofgzone, on=['name', 'year'], how='outer')
df = pd.merge(df, df_vtdef, on=['name', 'year'], how='outer')
df = pd.merge(df, df_tdef, on=['name', 'year'], how='outer')
df = pd.merge(df, df_opendef, on=['name', 'year'], how='outer')
df = pd.merge(df, df_wodef, on=['name', 'year'], how='outer')
df = pd.merge(df, df_0drib, on=['name', 'year'], how='outer')
df = pd.merge(df, df_1drib, on=['name', 'year'], how='outer')
df = pd.merge(df, df_2drib, on=['name', 'year'], how='outer')
df = pd.merge(df, df_3drib, on=['name', 'year'], how='outer')
df = pd.merge(df, df_7drib, on=['name', 'year'], how='outer')
df = pd.merge(df, df_cands, on=['name', 'year'], how='outer')
df = pd.merge(df, df_drives, on=['name', 'year'], how='outer')
df = pd.merge(df, df_shots, on=['name', 'year'], how='outer')
df = pd.merge(df, df_odef, on=['name', 'year'], how='outer')
df = pd.merge(df, df_2ptdef, on=['name', 'year'], how='outer')
df = pd.merge(df, df_3ptdef, on=['name', 'year'], how='outer')
df = pd.merge(df, df_defrim, on=['name', 'year'], how='outer')
df = pd.merge(df, df_odeflt6, on=['name', 'year'], how='outer')
df = pd.merge(df, df_odeflt10, on=['name', 'year'], how='outer')
df = pd.merge(df, df_odefgt15, on=['name', 'year'], how='outer')
df = pd.merge(df, df_pass, on=['name', 'year'], how='outer')
df = pd.merge(df, df_rebdet, on=['name', 'year'], how='outer')
df = pd.merge(df, df_boxout, on=['name', 'year'], how='outer')

###########################################
df = pd.merge(df, df_hustle, on=['name', 'year'], how='outer')

# Scale percentage quantities to be in range 0-1 (for convenience)
perc_cols = [col for col in df.columns if 'FGP' in col or '3PP' in col or 'FTP' in col or 'PERC' in col or 'PCT' in col or 'FREQ' in col]
df[perc_cols] = df[perc_cols].astype(float)/100.



In [110]:
#df = df.fillna(0)
df.to_csv("NBAAdvancedStats.csv")
print(df)

                    name  year   age     W     L  PTS_PT  FGM_PT  FGA_PT  \
0             AJ Hammons  2017  24.0   4.0  18.0    10.6     3.7     9.3   
1               AJ Price  2015  28.0  11.0  15.0    14.8     5.7    15.2   
2           Aaron Brooks  2015  30.0  50.0  32.0    18.2     6.6    15.6   
3           Aaron Brooks  2016  31.0  36.0  33.0    16.0     6.1    15.2   
4           Aaron Brooks  2017  32.0  36.0  29.0    13.0     4.9    12.1   
...                  ...   ...   ...   ...   ...     ...     ...     ...   
3003             Zhou Qi  2019  23.0   0.0   1.0    75.4    37.7    37.7   
3004        Zoran Dragic  2015  26.0   6.0  10.0    13.4     5.3    14.4   
3005      Zylan Cheatham  2020  24.0   1.0   1.0     3.6     1.8     7.3   
3006  Antonius Cleveland  2020   NaN   NaN   NaN     NaN     NaN     NaN   
3007            Kyle Guy  2020   NaN   NaN   NaN     NaN     NaN     NaN   

      FGP_PT  3PM_PT  ...  DEFL_PT  OFF_LB_RECOV_PT  DEF_LB_RECOV_PT  \
0      0.405   

In [2]:
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_rows')

In [1]:
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None
#df.columns.tolist()
